# Computo de alto desempeño

### Roberto Guerrero Bueno (rguerrero.mca16@lania.edu.mx)

In [1]:
import numpy as np
print np.__version__

import pandas as pd
print pd.__version__

import matplotlib as plt
print plt.__version__

import ipyparallel as ipp
print ipp.__version__

1.11.3
0.19.2
2.0.0
6.0.2


In [2]:
c = ipp.Client()

In [3]:
c.ids

[0, 1, 2, 3]

In [4]:
c

In [5]:
!ls ~/.ipython/profile_default/security

ipcontroller-client.json  ipcontroller-engine.json


In [6]:
cat ~/.ipython/profile_default/security/ipcontroller-engine.json

{
  "control": 57395, 
  "task": 51338, 
  "hb_ping": 35212, 
  "mux": 59724, 
  "pack": "json", 
  "hb_pong": 33164, 
  "ssh": "", 
  "key": "69db9eed-a956-4de5-8cdd-29a43b3bc5ff", 
  "registration": 52527, 
  "interface": "tcp://127.0.0.1", 
  "iopub": 55105, 
  "signature_scheme": "hmac-sha256", 
  "unpack": "json", 
  "location": "192.168.10.167"
}

In [7]:
cat ~/.ipython/profile_default/security/ipcontroller-client.json

{
  "control": 55893, 
  "task": 58752, 
  "notification": 45337, 
  "task_scheme": "leastload", 
  "mux": 36689, 
  "iopub": 45901, 
  "ssh": "", 
  "key": "69db9eed-a956-4de5-8cdd-29a43b3bc5ff", 
  "registration": 52527, 
  "interface": "tcp://127.0.0.1", 
  "signature_scheme": "hmac-sha256", 
  "pack": "json", 
  "unpack": "json", 
  "location": "192.168.10.167"
}

In [8]:
c[:]

<DirectView [0, 1, 2, 3]>

In [9]:
"Hola"

'Hola'

In [10]:
#Utiliza el motor 0 c[0].apply_sync
c[0].apply_sync(lambda:"Hola")

'Hola'

In [11]:
#Utilizar todos los motores c[:].apply_sync
c[:].apply_sync(lambda: "Hola a todos los motores")

['Hola a todos los motores',
 'Hola a todos los motores',
 'Hola a todos los motores',
 'Hola a todos los motores']

In [13]:
#creando un objeto DirectView para tener acceso desde un objeto a los motores
# puede ser dview = c[:] o dview = c.direct_view()
dview = c[:]

In [14]:
dview.apply_sync(lambda: "Hola a todos los motores accesando desde un objeto")

['Hola a todos los motores accesando desde un objeto',
 'Hola a todos los motores accesando desde un objeto',
 'Hola a todos los motores accesando desde un objeto',
 'Hola a todos los motores accesando desde un objeto']

In [15]:
dview

<DirectView [0, 1, 2, 3]>

In [16]:
#Obtener el ID del proceso del Sistema Operativo de cada motor
import os
dview.apply_sync(os.getpid)

[3153, 3154, 3155, 3156]

In [17]:
#Obtener el hostname de la maquina donde reside cada motor que tenemos
import socket
dview.apply_sync(socket.gethostname)

['lania-HP-Compaq-6720s',
 'lania-HP-Compaq-6720s',
 'lania-HP-Compaq-6720s',
 'lania-HP-Compaq-6720s']

In [19]:
#sumar 10 conjuntos de 3 numeros en paralelo usando todos los motores de la vista
dview.map_sync(lambda x,y,z:x+y+z,range(10),range(10),range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [21]:
#sumar 10 conjuntos de 3 numeros en paralelo usando motores alternados
c[::2].map_sync(lambda x,y,z:x+y+z,range(10),range(10),range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [22]:
#sumar 10 conjuntos de 3 numeros usando un solo motor
c[1].map_sync(lambda x,y,z:x+y+z,range(10),range(10),range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [23]:
#Load balanced view permite administrar la distribucion de tareas
lview = c.load_balanced_view()

In [24]:
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[500019.44347906468,
 500030.06323818234,
 499746.34065629949,
 499934.21333396441,
 499756.86087965767,
 500125.10488341888,
 499336.27089017828,
 499951.92854757968,
 500166.49613401905,
 499970.91003622179]

In [25]:
#Map(funcion,secuencia) aplica una funcion pasada a cada elemento de un objeto iterable y devuelve una lista que contiene todos los resultados de las llamadas de funcion

#funcion cube
def cube(x):
    return x**3

#Llamada a la funcion Cube
cube(2)

8

In [26]:
#Usar map "Manda a llamar la funcion cube con los numeros 1,2,3,4,5,6,7 y devuelve una lista
# de los resultados"
map(cube,range(1,7))

[1, 8, 27, 64, 125, 216]

In [29]:
#Otra forma de hacerlo sin usar map
lista = [cube(x) for x in range(1,7)]
lista

[1, 8, 27, 64, 125, 216]

In [31]:
#Devuelve una lista de un numero (a-70) elevado al cubo y lo guarda en la lista "serial"
serial = map(cube,range(1,70))

In [32]:
#Devuelve una lista de un numero (1-70) elevado al cubo y lo guarda en la lista "parallel"
parallel = dview.map_sync(cube,range(1,70))

In [33]:
#Revisa si las listas son iguales "serial == parallel"
serial == parallel

True

In [36]:
#10000 loops, best of 3: 37.5 µs per loop. Total: 375 ms
%timeit map(cube,range(1,70))
#10 loops, best of 3: 127 ms per loop. Total: 1270 ms
%timeit dview.map_sync(cube,range(1,70))

10000 loops, best of 3: 37.5 µs per loop
10 loops, best of 3: 127 ms per loop


In [37]:
#Definimos otra funcion de prueba
def multi(a,b):
    return a*b

#map_sync paraleliza una funcion
dview.map_sync(multi,[0,1,3,5],[2,4,5,8])

[0, 4, 15, 40]

In [38]:
#Definimos una funcion que determina si un numero dado es primo:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

In [39]:
PrimeQ(2)

True

In [40]:
PrimeQ(137)

True

In [41]:
PrimeQ(135)

False